In [2]:
import open_clip
import torch
from PIL import Image

# 加载 CLIP-ViT-B-16-DataComp
model, preprocess_train, preprocess = open_clip.create_model_and_transforms('hf-hub:laion/CLIP-ViT-B-16-DataComp.XL-s13B-b90K')
tokenizer = open_clip.get_tokenizer('hf-hub:laion/CLIP-ViT-B-16-DataComp.XL-s13B-b90K')

In [3]:
# 计算参数总量
total_params = sum(p.numel() for p in model.parameters())
print(f"模型参数总量: {total_params:,}")  # 逗号分隔便于阅读

模型参数总量: 149,620,737


In [12]:
model.eval()  # model in train mode by default, impacts some models with BatchNorm or stochastic depth active
image = preprocess(Image.open("motor.jpg")).unsqueeze(0)
text = tokenizer(["a car", "a people","a cat", "in elevator"])

with torch.no_grad(), torch.cuda.amp.autocast():
    image_features = model.encode_image(image)
    text_features = model.encode_text(text)
    image_features /= image_features.norm(dim=-1, keepdim=True)
    text_features /= text_features.norm(dim=-1, keepdim=True)

    text_probs = (100.0 * image_features @ text_features.T).softmax(dim=-1)

print("Label probs:", text_probs)  # prints: [[1., 0., 0.]]

Label probs: tensor([[7.8046e-06, 1.0589e-05, 2.3992e-06, 9.9998e-01]])
